In [ ]:
import pandas as pd
import sqlalchemy as sql
from sql_functions import get_sql_config
from sql_functions import get_engine
from sql_functions import push_to_database
from update import update_sql_table
import psycopg2

csv-File importieren.

In [ ]:
nrw_2022 = pd.read_csv("data/nrw_2022.csv")
nrw_2022.head(20)
nrw_2022.info()

Columns names anpassen, Sonderzeichen entfernen, in tausend Euro entfernen

In [ ]:
nrw_2022.columns = nrw_2022.columns.str.replace(" ", "_")
nrw_2022.columns = nrw_2022.columns.str.lower()
nrw_2022.rename(columns={"ansatz_2022_t€":"ansatz_2022_euro"}, inplace=True)
nrw_2022.tail()

Umbenennung der Namen im Columns

In [ ]:
nrw_2022.rename(columns = {'einzelplan':'ep', 'zählnummer':'counter'}, inplace = True)
nrw_2022.head()

droppen von columns

In [ ]:
nrw_2022.drop('ist_2020_in_euro', inplace=True, axis=1)

drop von column

In [ ]:
nrw_2022.drop('ansatz_2021_in_euro', inplace=True, axis=1)
nrw_2022.head()


drop column

In [ ]:
nrw_2022.drop('verpflichtungsermächtigung_in_euro', inplace=True, axis=1)
nrw_2022.head()

check-up

In [ ]:
nrw_2022

column eingefügt mit Jahr/Wert=2022

In [ ]:
nrw_2022.insert(loc=6,
          column='year',
          value=2022)

nrw_2022.head()

Column hinzugefügt mit state_id/wert= 10 (NRW)

In [ ]:
nrw_2022.insert(loc=7,
          column='state_id',
          value=10)

nrw_2022.head()

check-up

In [ ]:
nrw_2022

Reihenfolge nach upload_budget angepasst -> wird nocht geändert

In [ ]:
nrw_2022 = nrw_2022[['ep', 'kapitel', 'zweckbestimmung', 'ansatz_2022_in_euro', 'gruppe', 'counter', 'funktion', 'year', 'state_id']]
nrw_2022.head()

check-up

In [ ]:
nrw_2022.info()

Umwandlung der Werte im Columns um weiteres vorgehen zu ermöglichen 

In [ ]:
nrw_2022['ep'] = nrw_2022['ep'].astype(str)
nrw_2022['ep'] = nrw_2022['ep'].apply(lambda x:x.zfill(2))
nrw_2022['kapitel'] = nrw_2022['kapitel'].astype(str)
#sh_2022['titel'] = sh_2022['titel'].astype(str)
#sh_2022['titel'] = sh_2022['titel'].apply(lambda x:x.zfill(5))
nrw_2022['counter'] = nrw_2022['counter'].astype(str)
nrw_2022['counter'] = nrw_2022['counter'].apply(lambda x:x.zfill(2))
nrw_2022['funktion'] = nrw_2022['funktion'].astype(str)
nrw_2022['funktion'] = nrw_2022['funktion'].apply(lambda x:x.zfill(3))
nrw_2022

Kapitel überarbeitet mit nullziffer-> vergelich roh daten 

In [ ]:
nrw_2022['kapitel'] = nrw_2022['kapitel'].apply(lambda x:x.zfill(3))

In [ ]:
nrw_2022

Summen mit '.' werden ersetzt (tausender punkt)

In [ ]:
nrw_2022['ansatz_2022_in_euro'] = nrw_2022['ansatz_2022_in_euro'].str.replace(".", "")

check-up

In [ ]:
nrw_2022

umwandeln in int

In [ ]:
nrw_2022['ansatz_2022_in_euro'] = nrw_2022['ansatz_2022_in_euro'].astype(int)

checkp-up

In [ ]:
nrw_2022

alle Werte >=400 werden zu negative Zahlen umgewandelt

In [ ]:
nrw_2022.loc[nrw_2022['gruppe'] >= 400, 'ansatz_2022_in_euro'] = nrw_2022['ansatz_2022_in_euro']* -1
nrw_2022['gruppe'] = nrw_2022['gruppe'].astype(str)
nrw_2022['gruppe'] = nrw_2022['gruppe'].apply(lambda x:x.zfill(3))

letzter check up

In [ ]:
nrw_2022


In [ ]:
nrw_2022.rename(columns = {'funktion':'fkz', 'ansatz_2022_in_euro':'amount'}, inplace = True)

In [ ]:
nrw_2022.head()

In [ ]:
get_sql_config()
engine = get_engine()

In [ ]:
#push_to_database(nrw_2022, "budget_all", engine, "capstone_public_budgeting")

In [ ]:
df_ep_kapitel = pd.read_csv('data/ep_kapitel_nrw_22.csv',encoding= 'UTF-8',dtype={'ep':str,'kapitel':str})

In [ ]:
df_ep_kapitel['year'] = 2022
df_ep_kapitel['state_id'] = 10
df_ep_kapitel

In [ ]:
update_sql_table(df_ep_kapitel[['ep','kapitel','kapitel_txt','year','state_id']],'kapitel')
update_sql_table(df_ep_kapitel[['ep','ep_txt','year','state_id']],'einzelplaene')

Wird eigentlich nicht mehr benötigt

In [ ]:
#push_to_database(df_ep_kapitel[['ep','kapitel','kapitel_txt','year','state_id']], "kapitel", engine, "capstone_public_budgeting")

In [ ]:
#push_to_database(df_ep_kapitel[['ep','ep_txt','year','state_id']].drop_duplicates(),
#                "einzelplaene",
#                engine, 
#                "capstone_public_budgeting"
#                )